In [6]:
import linktransformer as lt


In [10]:
from dotenv import load_dotenv
load_dotenv()
import os
import pandas as pd

In [5]:
# folder_path = "/home/njui/kn_workspace/curriculum_taxonomy_extractor/data/processed/CBC Math/Grade 4 Curriculum designs Maths CSV"

In [15]:
import os
import chardet
import pandas as pd

# Specify multiple folder paths here
folder_paths = [
    "../data/processed/cbc_math/Grade 3 Curriculum designs Maths CSV",
    "../data/processed/cbc_math/Grade 4 Curriculum designs Maths CSV",
    "../data/processed/cbc_math/Grade 6 Curriculum designs Maths CSV",
    # Add more paths as needed
]

# Initialize lists to store the data
file_names = []
file_paths = []
folder_names = []
file_details = []
indicator_files = []
read_status= []

# Keywords to check in the CSV contents
key_words = ["meets expectations", "below expectations"]

# Iterate through each specified folder path
for folder_path in folder_paths:
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            # Construct the full file path
            full_file_path = os.path.join(root, file)
            file_names.append(file)
            file_paths.append(full_file_path)
            folder_name = os.path.basename(root)
            folder_names.append(folder_name)

            # Read the CSV file and convert it to a JSON string
            try:
                with open(full_file_path, 'rb') as file:
                    result = chardet.detect(file.read())
                    encoding = result['encoding']
                df_temp = pd.read_csv(full_file_path, encoding=encoding)
                details = df_temp.to_json()
                file_details.append(details)
                
                # Check if any of the key words are in the CSV
                indicator = any(word in df_temp.to_string().lower() for word in key_words)
                indicator_files.append(indicator)
                read_status.append(True)
            except Exception as e:
                file_details.append({'error': str(e)})
                read_status.append(False)
                indicator_files.append('')  # Default to False in case of error

# Create a dataframe with the collected data
df = pd.DataFrame({
    'file_name': file_names,
    'file_path': file_paths,
    'folder_name': folder_names,
    'details': file_details,
    'indicator_status': indicator_files,
    'read_status': read_status
})


In [18]:
# df.head(10)

In [17]:
distinct_read_status = df['read_status'].unique()
print(distinct_read_status)

[ True]


In [19]:
df_grouped = df.groupby('indicator_status')

# Get the DataFrames for each group
df_rubrics = df_grouped.get_group(True)
df_strands = df_grouped.get_group(False)


In [40]:
# df_strands.head

In [20]:
# Using the lt.merge function to perform a semantic merge
df_merged = lt.merge(
    df1=df_strands, 
    df2=df_rubrics, 
    on=['file_name','folder_name','details'], 
    merge_type='1:1',
    model='all-MiniLM-L6-v2',
    left_on=None, 
    right_on=None, 
    suffixes=('_x', '_y'), 
    use_gpu=False, 
    batch_size=128, 
    openai_key=os.getenv('OPENAI_API_KEY')  # Assuming you've set your API key in the environment
)


No base organization specified, if there is an error, it is likely because of that.
Trying to append the model : sentence-transformers/all-MiniLM-L6-v2 and linktransformers/all-MiniLM-L6-v2. Check your path otherwise!


Trying sentence-transformers/all-MiniLM-L6-v2...
No base organization specified, if there is an error, it is likely because of that.
Trying to append the model : sentence-transformers/all-MiniLM-L6-v2 and linktransformers/all-MiniLM-L6-v2. Check your path otherwise!
Trying sentence-transformers/all-MiniLM-L6-v2...
2024-03-05 16:26:21 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-05 16:26:22 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-05 16:26:22 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-05 16:26:22 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-05 16:26:23 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-05 16:26:23 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-05 16:26:24 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-05 1

In [22]:
df_merged.shape

(48, 15)

In [32]:
df_merged.to_csv('../data/processed/df_merged_math_cbc.csv', index=False)


In [11]:
df_merged = pd.read_csv('../data/processed/df_merged_math_cbc.csv')

In [4]:
######### Structured output from details_x and details_y

from pydantic import BaseModel
from typing import List

# from llama_index.program.openai import OpenAIPydanticProgram



In [3]:
from pydantic import BaseModel
from typing import List

from llama_index.program.openai import OpenAIPydanticProgram

In [7]:
from llama_index.llms.openai import OpenAI

In [5]:
from pydantic import BaseModel
from typing import List

class Rubric(BaseModel):
    """
    A rubric outlines specific criteria used to assess a student's performance on tasks related to a sub-strand. Each rubric includes levels of expectations such as 'Exceeds Expectations', 'Meets Expectations', 'Approaches Expectations', and 'Below Expectations', providing clear benchmarks for assessment.
    """
    indicator_name: str
    exceeds_expectations: str
    meets_expectations: str
    approaches_expectations: str
    below_expectations: str

class SubStrand(BaseModel):
    """
    A sub-strand is a component of a broader curriculum strand that focuses on a specific topic or set of skills within the subject area. It includes specific learning outcomes that students are expected to achieve, key inquiry questions to guide exploration and understanding, suggested learning experiences to achieve these outcomes, and rubrics for evaluating student progress.
    """
    sub_strand_name: str
    specific_learning_outcomes: str
    key_inquiry_questions: str
    suggested_learning_experiences: str
    # rubrics: List[Rubric]

class Strand(BaseModel):
    """
    Represents a key component of the curriculum for a specific grade and subject area. A strand is comprised of multiple sub-strands, each addressing different topics or areas of knowledge within the subject. This model organizes the curriculum content by grade, subject, and the names of the strands and includes the detailed breakdown of sub-strands for comprehensive curriculum coverage.
    """
    grade: str
    subject: str
    strand_name: str
    sub_strands: List[SubStrand]


In [12]:
program = OpenAIPydanticProgram.from_defaults(
    output_cls=Strand,  # The class we're aiming to fill with GPT-3's response
    llm=OpenAI(temperature=0, model="gpt-3.5-turbo"),  # Adjust model as needed
    prompt_template_str="""\
        Given the details from a curriculum document in JSON format, extract the relevant information to fill out the structure of educational content. The JSON data contains details about curriculum strands, sub-strands, and their respective learning outcomes, inquiry questions, suggested learning experiences, and rubrics for assessment.

        For the grade and subject, extract them from the folder names. For sub-strands, look for sections within the strand. For each sub-strand, identify its specific learning outcomes, key inquiry questions, and suggested learning experiences. Finally, detail the rubrics for evaluating the sub-strand's learning outcomes.

        Grade and Subject from folder name: {input_folder_name}
        Strand and Sub-Strand details in JSON: {input_details_x}
        Rubrics in JSON: {input_details_y}

        Structure the extracted information as follows:
        - Grade:
        - Subject:
        - Strand Name:
        - List of Sub-Strands, each with:
            - Sub-Strand Name:
            - Specific Learning Outcomes:
            - Key Inquiry Questions:
            - Suggested Learning Experiences:
            - Rubrics: (List, detailing indicators and expectations levels)

        Please provide the structured output based on the provided information.
        """,
    verbose=True,
    allow_multiple=True
)

In [13]:
strands = []
# Assuming `df_merged` is loaded and `program` is configured as shown earlier

for index, row in df_merged.iterrows():
    grade, subject = row['folder_name_x'].split()[:2]  # Simplified extraction, adjust as necessary
    input_details_x = row['details_x']
    input_details_y = row['details_y']
    
    # # Construct the input for the model
    # formatted_input = prompt_template_str.format(
    #     input_folder_name=f"{grade} {subject}",
    #     input_details_x=input_details_x,
    #     input_details_y=input_details_y
    # )
    
    # Execute the program (This step is theoretical in this context)
    strand = program(input_folder_name=f"{grade} {subject}",input_details_x=input_details_x,input_details_y=input_details_y )  # Modify as per actual function call syntax
    strands.append(strand)


Function call: Strand with args: {"grade":"Grade 3","subject":"Mathematics","strand_name":"Measurement","sub_strands":[{"sub_strand_name":"Time","specific_learning_outcomes":"By the end of the sub-strand, the learner should be able to:\n\na. Identify the minute as a unit of measuring time,\nb. Read and tell time using the digital clock,\nc. Read and tell time using ‘past’ and ‘to’ the hour using the clock face,\nd. Write time using ‘past’ and ‘to’ the hour,\ne. Estimate time in hours,\nf. Add and subtract time involving hours and minutes without conversion in real life","key_inquiry_questions":"How do we convert hours to minutes","suggested_learning_experiences":"- Learners to discuss the divisions on a clock face and what each division represents.\n- Learners to read time on a digital clock\n- Learners in pairs/groups to discuss the relationship between hours and minutes using a clock face.\n- Learners in pairs/groups to read, tell and write time using ‘past’ and ‘to’ the hour.\n- Lea

In [14]:
# strands[0]

[Strand(grade='Grade 3', subject='Mathematics', strand_name='Measurement', sub_strands=[SubStrand(sub_strand_name='Time', specific_learning_outcomes='By the end of the sub-strand, the learner should be able to:\n\na. Identify the minute as a unit of measuring time,\nb. Read and tell time using the digital clock,\nc. Read and tell time using ‘past’ and ‘to’ the hour using the clock face,\nd. Write time using ‘past’ and ‘to’ the hour,\ne. Estimate time in hours,\nf. Add and subtract time involving hours and minutes without conversion in real life', key_inquiry_questions='How do we convert hours to minutes', suggested_learning_experiences='- Learners to discuss the divisions on a clock face and what each division represents.\n- Learners to read time on a digital clock\n- Learners in pairs/groups to discuss the relationship between hours and minutes using a clock face.\n- Learners in pairs/groups to read, tell and write time using ‘past’ and ‘to’ the hour.\n- Learners in pairs/groups to es

In [12]:
# df_merged.head(1)

NameError: name 'df_merged' is not defined

In [ ]:
# grade: [Grade Level from folder_name_x ]
# subject: [Subject Name from folder_name_x]
# sub_strand: [Name of each sub-strand within a strand. Found in details_x]
# specific_learning_outcomes: [List of learning outcomes for each sub-strand. Found in details_x]
# key_inquiry_question: [Key questions for each sub-strand. Found in details_x]]
# suggested_learning_experiences: [List of suggested activities and experiences. Found in details_x]
# assessment_rubrics: Array of objects, each representing an assessment indicator with detailed rubrics. For each indicator, include:
# indicator_name: Name of the indicator.
# rubrics: Array of objects detailing the achievement levels (Exceeds, Meets, Approaches, Below Expectations), with a level key and a statement key describing the criteria for each level.